In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as ntx

In [2]:
file_input="input_test.txt"
#file_input="input.txt"

In [3]:
with open(file_input,'r') as f:
    lines = f.readlines()

In [4]:
from collections import deque


In [5]:
valves = {}
tunnels = {}

for line in lines:
    line = line.strip()
    valve = line.split()[1]
    flow = int(line.split(";")[0].split("=")[1])
    targets = line.split("to ")[1].split(" ", 1)[1].split(", ")
    valves[valve] = flow
    tunnels[valve] = targets


In [6]:
valves

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

In [7]:
tunnels

{'AA': ['DD', 'II', 'BB'],
 'BB': ['CC', 'AA'],
 'CC': ['DD', 'BB'],
 'DD': ['CC', 'AA', 'EE'],
 'EE': ['FF', 'DD'],
 'FF': ['EE', 'GG'],
 'GG': ['FF', 'HH'],
 'HH': ['GG'],
 'II': ['AA', 'JJ'],
 'JJ': ['II']}

In [8]:
dists = {} 
nonempty = []

In [9]:
#Builds up a dictionary with distances to any other valves
for valve in valves:

    if valve != "AA":
        nonempty.append(valve)
        #Build up a ordered list of valves that are nonempty,
        #For some reason excludes 'AA'

    dists[valve] = {valve: 0, "AA": 0}
    visited = {valve}
    
    queue = deque([(0, valve)])
    
    while queue:
        distance, position = queue.popleft()
        for neighbor in tunnels[position]:
            if neighbor in visited:
                continue
            visited.add(neighbor)
            if valves[neighbor]:
                dists[valve][neighbor] = distance + 1
            queue.append((distance + 1, neighbor))

    del dists[valve][valve]
    if valve != "AA":
        del dists[valve]["AA"]


In [10]:
dists

{'AA': {'DD': 1, 'BB': 1, 'CC': 2, 'EE': 2, 'JJ': 2, 'HH': 5},
 'BB': {'CC': 1, 'DD': 2, 'EE': 3, 'JJ': 3, 'HH': 6},
 'CC': {'DD': 1, 'BB': 1, 'EE': 2, 'JJ': 4, 'HH': 5},
 'DD': {'CC': 1, 'EE': 1, 'BB': 2, 'JJ': 3, 'HH': 4},
 'EE': {'DD': 1, 'CC': 2, 'HH': 3, 'BB': 3, 'JJ': 4},
 'FF': {'EE': 1, 'DD': 2, 'HH': 2, 'CC': 3, 'BB': 4, 'JJ': 5},
 'GG': {'HH': 1, 'EE': 2, 'DD': 3, 'CC': 4, 'BB': 5, 'JJ': 6},
 'HH': {'EE': 3, 'DD': 4, 'CC': 5, 'BB': 6, 'JJ': 7},
 'II': {'JJ': 1, 'DD': 2, 'BB': 2, 'CC': 3, 'EE': 3, 'HH': 6},
 'JJ': {'DD': 3, 'BB': 3, 'CC': 4, 'EE': 4, 'HH': 7}}

In [11]:
nonempty

['BB', 'CC', 'DD', 'EE', 'FF', 'GG', 'HH', 'II', 'JJ']

In [12]:
#Seetup indices to each of the nonzero valves
indices = {}

for index, element in enumerate(nonempty):
    indices[element] = index

In [13]:
indices

{'BB': 0,
 'CC': 1,
 'DD': 2,
 'EE': 3,
 'FF': 4,
 'GG': 5,
 'HH': 6,
 'II': 7,
 'JJ': 8}

In [14]:
cache = {}

#dictionary of items (time,valve,bitmask):maxval

In [33]:
cache = {}
def dfs(time, valve, bitmask):
    #Gets the maximim score starting from a given valve, timeleft and given opened gas valves

    if (time, valve, bitmask) in cache:
        #If already calculated, just use calculation
        return cache[(time, valve, bitmask)]
    
    maxval = 0
    for neighbor in dists[valve]: #Check all useful neighbours from the valve
        bit = 1 << indices[neighbor] #Convert the neighbor to a binary

        if bitmask & bit: 
            continue #check if valve has been opened? if yes, try another valve destination

        remtime = time - dists[valve][neighbor] - 1 #remaining time after reaching new valve
        if remtime <= 0:
            continue #Check if remaining time is below zero. if yes try another neigbor
        
        #maxval = max(maxval, dfs(remtime, neighbor, bitmask | bit) + valves[neighbor] * remtime) #Split below
        newdfs = dfs(remtime, neighbor, bitmask | bit)
        totpress = newdfs + valves[neighbor] * remtime #Calculates the new total pressure at final if this is visited and opened
        maxval = max(maxval, totpress)
        
    cache[(time, valve, bitmask)] = maxval
    return maxval

In [34]:
dfs(30,'AA',0)

1651